In [1]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from catboost import Pool, CatBoostClassifier
import numpy as np
import re
from string import punctuation
from sklearn.metrics import roc_curve, auc

            +------------+     +---------------+     +-----------+
            | user_data  |     | post_text_df  |     | feed_data |
            +------------+     +---------------+     +-----------+
            | age        |     | id            |     | timestamp |
            | city       |     | text          |     | user_id   |
            | country    |     | topic         |     | post_id   |
            | exp_group  |     +---------------+     | action    |
            | gender     |           7,023           | target    |
            | id         |                           +-----------+
            | os         |                             76,892,800
            | source     |
            +------------+
                163,205 

In [2]:
user_data = pd.read_csv('C:\\Users\\Alex\\Desktop\\data\\user_data.csv')  # Загрузка данных пользователя
post_text_df = pd.read_csv('C:\\Users\\Alex\\Desktop\\data\\post_text_df.csv')  # Загрузка текстовых данных поста
feed_data = pd.read_csv('C:\\Users\\Alex\\Desktop\\data\\feed_data.csv')  # Загрузка данных кормления


In [18]:
user_data

,user_id,age
0,200,34
1,201,37
2,202,17
3,203,18
4,204,36
...,...,...
163200,168548,36
163201,168549,18
163202,168550,41
163203,168551,38


In [3]:
# Переименование столбцов идентификаторов
user_data = user_data.rename(columns={'id': 'user_id'})
post_text_df = post_text_df.rename(columns={'id': 'post_id'})

# Формируем столбец "text_len"
post_text_df['text_len'] = post_text_df['text'].apply(len)

post_text_df = post_text_df.drop('text', axis=1)

user_data = user_data.drop(['country', 'city', 'exp_group', 'os', 'source', 'gender'], axis=1)

# Category encoding for topic colum
le = LabelEncoder()
post_text_df['topic_encoded'] = le.fit_transform(post_text_df['topic'])

# Объединение таблиц
data = feed_data.merge(user_data, on='user_id', how='left')
data = data.merge(post_text_df, on='post_id', how='left')


In [4]:
# Let's create a subset dataframe with only 'view' actions
views_df = data[data['action'] == 'view']

# Now, let's count views per post
views_per_post = views_df['post_id'].value_counts()

# Let's create a subset dataframe with only 'like' actions
likes_df = data[data['action'] == 'like']

# Now, let's count likes per post
likes_per_post = likes_df['post_id'].value_counts()

# Now, let's merge these two series into a new dataframe
post_stats = pd.DataFrame({
    'views': views_per_post,
    'likes': likes_per_post
})

# Replace NaN values with 0 (assuming that NaN means there were no likes/views)
post_stats.fillna(0, inplace=True)

# Let's calculate the likes percentage for each post from all likes
total_likes = post_stats['likes'].sum() # Total likes across all posts
post_stats['like_percentage'] = (post_stats['likes'] / total_likes) * 100

# Let's reset the index so 'post_id' becomes a column
post_stats.reset_index(inplace=True)
post_stats.rename(columns={'index': 'post_id'}, inplace=True)

# Now we merge this dataframe with the original one, on 'post_id'
# 'left' ensures that all rows in the original data are kept, even if they don't have a match in post_stats
data = pd.merge(data, post_stats, on='post_id', how='left')

In [10]:
data.head(20)

,timestamp,user_id,post_id,action,target,age,topic,text_len,topic_encoded,views,likes,like_percentage
0,2021-12-26 16:40:07,52431,6920,view,0,34,movie,1148,3,6610,717,0.008737
1,2021-12-26 16:42:42,52431,3840,view,0,34,covid,140,1,22496,2842,0.034632
2,2021-12-26 16:45:27,52431,1716,view,0,34,sport,2360,5,22072,2807,0.034205
3,2021-12-26 16:46:39,52431,1054,view,0,34,politics,1908,4,12219,871,0.010614
4,2021-12-26 16:48:54,52431,963,view,0,34,politics,2489,4,22136,2804,0.034169
5,2021-12-26 16:50:21,52431,1219,view,0,34,politics,2122,4,22236,2771,0.033767
6,2021-12-26 16:51:05,52431,971,view,0,34,politics,3032,4,22043,2857,0.034815
7,2021-12-26 16:52:19,52431,974,view,0,34,politics,3796,4,17442,1574,0.019180
8,2021-12-26 16:55:13,52431,2042,view,0,34,tech,1846,6,12039,833,0.010151
9,2021-12-26 16:56:39,52431,3950,view,0,34,covid,103,1,11977,923,0.011247


In [16]:
# collect and save all unique user ids


['view' 'like']


In [11]:
# Выбираем строки, где действие равно "like"
likes_df = data[data['action'] == 'like']

# Находим уникальных пользователей, которые сделали хотя бы один лайк
users_with_likes = likes_df['user_id'].nunique()

# Находим общее количество уникальных пользователей в датасете
total_users = data['user_id'].nunique()

# Вычисляем долю пользователей, которые сделали хотя бы один лайк
users_with_likes_percentage = (users_with_likes / total_users) * 100

print(f"Доля пользователей, которые сделали хотя бы один лайк: {users_with_likes_percentage:.2f}%")


Доля пользователей, которые сделали хотя бы один лайк: 100.00%


In [12]:
users_with_likes

163202

In [13]:
total_users

163205

In [17]:
# Группируем данные по user_id и суммируем лайки и просмотры
user_sums = data.groupby('user_id')[['likes', 'views']].sum().reset_index()

# Отображаем результат
print(user_sums)

        user_id    likes    views
0           200   635373  5427310
1           201  1215047  9541377
2           202  1289648  9739379
3           203   672409  5169532
4           204   269183  2092592
...         ...      ...      ...
163200   168548   614829  4845396
163201   168549   482719  3682347
163202   168550   544282  4408358
163203   168551   851236  6584036
163204   168552   461395  3472872

[163205 rows x 3 columns]


In [14]:
# list unique user ids
